# Histogramm of price differences before and after energy crisis

## Set up

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from scripts.utils import scale_font_latex
from scripts.countries import ENERGY_CRISIS, country_codes

In [3]:
TRUE_START = pd.Timestamp("20180101T00", tz="utc")
TRUE_END = pd.Timestamp("20231231T23", tz="utc")
true_years = f"{TRUE_START.year}-{TRUE_END.year}"
COUNTRY_CODE = "FR"


data = pd.read_csv(
    f"../data/processed/combined_data/data_full_{COUNTRY_CODE}_{true_years}.csv",
    parse_dates=["timestamp"],
).set_index("timestamp")
data.index = pd.to_datetime(data.index, utc="utc")


# European prices da
price = pd.read_csv(
    "../data/processed/european_prices_da.csv", parse_dates=["timestamp"]
).set_index("timestamp")
price = price.truncate(before=TRUE_START, after=TRUE_END)
zones = [cc for cc in country_codes if cc not in ["IT_ROSN", "IT_CALA"]]

## Histogramm of price difference before and after energy crisis

In [ ]:
def plot_price_diff_hist(price_1, price_2, ax, country):
    """
    Plot a histogram of price differences (price_1 - price_2) before and after the energy crisis.

    Parameters:
    price_1 (pandas.Series): The first price series.
    price_2 (pandas.Series): The second price series.
    ax (matplotlib.axes.Axes): The axes on which to plot the histogram.

    Returns:
    None
    """
    price_diff = pd.DataFrame(index=pd.date_range(TRUE_START, TRUE_END, freq="h"))
    price_diff.index.name = "timestamp"
    price_diff["price_diff"] = price_1 - price_2

    colors = sns.color_palette("colorblind")

    bin_min = -250
    bin_max = 250
    bins = np.arange(bin_min, bin_max + 1, 1)  # Create an array of bin edges
    price_differences = [
        price_diff.loc[TRUE_START:ENERGY_CRISIS],
        price_diff.loc[ENERGY_CRISIS:TRUE_END],
    ]
    labels = ["before", "during"]
    for i, price_diff_selected in enumerate(price_differences):
        counts, bin_edges = np.histogram(
            price_diff_selected["price_diff"], bins=bins, density=False
        )
        # Calculate the center of each bin
        bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
        # Creating a basic step plot
        ax.step(
            bin_centers,
            counts,
            where="mid",
            color=colors[i],
            label=country + ": " + labels[i],
        )
        price_diff_selected_mean = np.mean(price_diff_selected["price_diff"])
        # Plot the mean as a vertical line
        ax.axvline(
            price_diff_selected_mean, color=colors[i], linestyle="dashed", linewidth=2
        )
        print(
            "Mean"
            + " "
            + country
            + " "
            + labels[i]
            + ":"
            + str(price_diff_selected_mean)
        )
        print(
            "Sync"
            + " "
            + country
            + " "
            + labels[i]
            + ":"
            + str(np.count_nonzero(price_diff_selected["price_diff"] == 0))
        )
        ax.set_ylim(0, 1600)

    ax.legend()
    ax.set_ylabel("Counts")
    scale_font_latex(1.5)
    ax.set_xlim(-60, 60)


fig, axes = plt.subplots(4, 1, figsize=(16, 16), sharex=True)
plot_price_diff_hist(price["FR"], price["IT_NORD"], axes[0], "IT-North")
plot_price_diff_hist(price["FR"], price["DE_LU"], axes[1], "DE LU")
plot_price_diff_hist(price["FR"], price["ES"], axes[2], "ES")
plot_price_diff_hist(price["FR"], price["BE"], axes[3], "BE")
plt.xlabel("Price difference (EUR)")

In [ ]:
def plot_price_diff(price_1, price_2, color, ax, label):
    """
    Plot the price difference (price_1 - price_2) between two price series over time.

    Parameters:
    price_1 (pandas.Series): The first price series.
    price_2 (pandas.Series): The second price series.
    color (str): The color for the plot.
    ax (matplotlib.axes.Axes): The axes on which to plot the price difference.
    label (str): The label for the plot legend.

    Returns:
    None
    """
    price_diff = pd.DataFrame(index=pd.date_range(TRUE_START, TRUE_END, freq="h"))
    price_diff.index.name = "timestamp"
    price_diff["price_diff"] = price_1 - price_2
    price_diff["rel_price_diff"] = 2 * (price_1 - price_2) / (price_1 + price_2)
    price_diff.replace(
        [np.inf, -np.inf], np.nan, inplace=True
    )  # Replace infinities with NaN
    price_diff.dropna(inplace=True)  # Drop rows with NaN values
    price_diff_std = price_diff.resample("1ME").std()["price_diff"]
    price_diff_mean = price_diff.resample("1ME").mean()["price_diff"]

    # Plot the monthly mean
    ax.plot(
        price_diff_mean.index,
        price_diff_mean,
        marker="o",
        label=label,
        color=color,
    )

    # Fill between the mean ± std
    ax.fill_between(
        price_diff_mean.index,
        price_diff_mean - price_diff_std,
        price_diff_mean + price_diff_std,
        color=color,
        alpha=0.3,
    )
    # Add a horizontal dashed line at y = 0
    ax.axhline(0, color="black", linewidth=1)
    # Add a vertical line at energy crisis
    ax.axvline(x=ENERGY_CRISIS, color="black", linestyle="dashed", linewidth=1)
    ax.legend(loc="upper left")
    ax.set_ylabel("Price diff. (EUR)")
    ax.set_xlim(TRUE_START, TRUE_END)


def plot_export_balance(export, color, ax, label):
    ax.set_ylabel("Export (MW)")
    ax.plot(export.resample("1ME").mean(), label=label, marker="o", color=color)
    ax.legend(loc="upper left")
    ax.fill_between(
        export.resample("1ME").mean().index,
        export.resample("1ME").mean() - export.resample("1ME").std(),
        export.resample("1ME").mean() + export.resample("1ME").std(),
        color=color,
        alpha=0.3,
    )
    # Add a vertical line at energy crisis
    ax.axvline(x=ENERGY_CRISIS, color="black", linestyle="dashed", linewidth=1)
    # Add a horizontal dashed line at y = 0
    ax.axhline(0, color="black", linewidth=1)
    ax.set_xlim(TRUE_START, TRUE_END)


scale_font_latex(1.5)
colors = sns.color_palette("colorblind")
fig, axes = plt.subplots(4, 1, figsize=(16, 18), sharex=True)
plot_price_diff(price["FR"], price["DE_LU"], color=colors[0], ax=axes[0], label="DE-LU")
plot_price_diff(price["FR"], price["BE"], color=colors[1], ax=axes[1], label="BE")
plot_price_diff(price["FR"], price["ES"], color=colors[2], ax=axes[2], label="ES")
plot_price_diff(
    price["FR"], price["IT_NORD"], color=colors[3], ax=axes[3], label="IT-North"
)
plt.xlabel("Time")
fig_dir = "../reports/figures/price_diff"
os.makedirs(fig_dir, exist_ok=True)
fig.savefig(fig_dir + f"/price_diff_FR.pdf", bbox_inches="tight")
plt.show()

colors = sns.color_palette("colorblind")
fig, axes = plt.subplots(4, 1, figsize=(16, 18), sharex=True)
plot_export_balance(data["FR->DE_LU"], color=colors[0], ax=axes[0], label="DE-LU")
plot_export_balance(data["FR->BE"], color=colors[1], ax=axes[1], label="BE")
plot_export_balance(data["FR->ES"], color=colors[2], ax=axes[2], label="ES")
plot_export_balance(data["FR->IT_NORD"], color=colors[3], ax=axes[3], label="IT-North")
plt.xlabel("Time")